## Make the velocity vehicle DATABASE

In [37]:
import configparser
# Define DB credentials
# Read database credentials from db.properties
import mysql.connector
config = configparser.ConfigParser()
config.read('db.properties')

db_config = {
    'host': config.get('mysql', 'host'),
    'user': config.get('mysql', 'user'),
    'password': config.get('mysql', 'password')
}

In [38]:
# Connect to MySQL
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Drop the schema if it exists
cursor.execute("DROP DATABASE IF EXISTS Velocity_vehicles")
# Create the database
cursor.execute("CREATE DATABASE IF NOT EXISTS Velocity_vehicles")
cursor.execute("USE Velocity_vehicles")

### make the car_brands table

In [39]:
# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS car_brands")

# Create the car_brands table
cursor.execute("""
    CREATE TABLE car_brands (
        car_brand VARCHAR(50) NOT NULL,
        brand_id VARCHAR(255) PRIMARY KEY
    )
""")

In [40]:
import pandas as pd

# Read data from car_brands.xlsx and insert into car_brands table
try:
    df = pd.read_excel('.\\data\\car_brands.xlsx')
    for _, row in df.iterrows():
        try:
            cursor.execute(
                "INSERT INTO car_brands (car_brand, brand_id) VALUES (%s, %s)",
                (row['make'], row['brand_id'])
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

### make the dealers table

In [41]:
cursor.execute("DROP TABLE IF EXISTS dealers")

# Create the dealers table
cursor.execute("""
    CREATE TABLE dealers (
        name VARCHAR(100) NOT NULL,
        brand_id VARCHAR(255),
        password VARCHAR(255) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        phone VARCHAR(10) NOT NULL UNIQUE,
        address VARCHAR(255) NOT NULL,
        city VARCHAR(100) NOT NULL,
        pincode INT NOT NULL,
        age INT NOT NULL check ((age<=70) and (age>=18)),
        gender VARCHAR(10) NOT NULL,
        description TEXT NOT NULL,
        username VARCHAR(100) PRIMARY KEY,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id)
    )
""")

In [42]:
# Read data from dealers.xlsx and insert into dealers table
try:
    df_dealers = pd.read_excel('.\\data\\dealers.xlsx')
    for _, row in df_dealers.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO dealers (
                    name, brand_id, password, email, phone, address, city, pincode, age, gender, description, username
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Name'],
                    row['Brand_ID'],
                    row['password'],
                    row['Email'],
                    row['Phone'],
                    row['Address'],
                    row['City'],
                    int(row['pincode']),
                    int(row['Age']),
                    row['gender'],
                    row['description'],
                    row['userName']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### make the Customer table

In [43]:
# Drop the table if it exists
cursor.execute("DROP TABLE IF EXISTS customers")

# Create the customers table
cursor.execute("""
    CREATE TABLE customers (
        name VARCHAR(100) NOT NULL,
        password VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        phone VARCHAR(10) NOT NULL UNIQUE,
        address VARCHAR(255) NOT NULL,
        city VARCHAR(100) NOT NULL,
        pincode INT NOT NULL,
        gender VARCHAR(10) NOT NULL,
        age INT check ((age<=70) and (age>=18)),
        username VARCHAR(100) PRIMARY KEY
    )
""")

In [44]:
# Read data from customers.xlsx and insert into customers table
try:
    df_customers = pd.read_excel('.\\data\\customers.xlsx')
    for _, row in df_customers.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO customers (
                    name, password, email, phone, address, city, pincode, gender, age, username
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Name'],
                    row['password'],
                    row['Email'],
                    str(row['Phone']),
                    row['Address'],
                    row['City'],
                    int(row['Pincode']),
                    row['Gender'],
                    int(row['Age']),
                    row['userName']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### cars table


In [45]:
# Drop the cars table if it exists
cursor.execute("DROP TABLE IF EXISTS cars")

# Create the cars table with boolean columns for the specified fields
cursor.execute("""
    CREATE TABLE cars (
        car_id VARCHAR(255) PRIMARY KEY,
        brand_id VARCHAR(255),
        dealer_username VARCHAR(100),
        model VARCHAR(100) NOT NULL,
        variant VARCHAR(100) NOT NULL,
        rating INT,
        price_inr DECIMAL(15, 2) NOT NULL,
        engine_cc INT NOT NULL,
        cylinders INT NOT NULL CHECK ((cylinders>=2) AND (cylinders<=8)),
        fuel_capacity INT NOT NULL,
        fuel_type VARCHAR(100) NOT NULL,
        body_type VARCHAR(100) NOT NULL,
        mileage FLOAT NOT NULL CHECK ((mileage>=7.8) AND (mileage<=45)),
        gears INT NOT NULL CHECK ((gears>=4) AND (gears<=9)),
        power_steering VARCHAR(100) NOT NULL,
        keyless_entry BOOLEAN NOT NULL,
        power_hp FLOAT NOT NULL,
        torque_nm FLOAT NOT NULL,
        seating_capacity INT NOT NULL CHECK ((seating_capacity>=2) AND (seating_capacity<=16)),
        transmission_type VARCHAR(100) NOT NULL,
        warranty INT NOT NULL,
        sun_visor BOOLEAN NOT NULL,
        ventilation_system BOOLEAN NOT NULL,
        abs BOOLEAN NOT NULL,
        airbags BOOLEAN NOT NULL,
        color VARCHAR(100) NOT NULL,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id),
        FOREIGN KEY (dealer_username) REFERENCES dealers(username)
    )
""")

In [46]:
# Read data from cars.xlsx and insert into cars table
try:
    df_cars = pd.read_excel('.\\data\\cars.xlsx')
    for _, row in df_cars.iterrows():
        try:
            cursor.execute(
                """
                INSERT INTO cars (
                    car_id, brand_id, dealer_username, model, variant,rating, price_inr, engine_cc, cylinders,
                    fuel_capacity, fuel_type, body_type, mileage, gears, power_steering, keyless_entry,
                    power_hp, torque_nm, seating_capacity, transmission_type, warranty, sun_visor,
                    ventilation_system, abs, airbags, color
                ) VALUES (
                    %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['Car_id'],
                    row['Brand_ID'],
                    row['DealerUsername'],
                    row['model'],
                    row['variant'],
                    row['rating'],
                    float(row['Price_INR']),
                    int(row['engine_cc']),
                    int(row['Cylinders']),
                    int(row['fuel_capacity']),
                    row['Fuel_type'],
                    row['Body_type'],
                    float(row['mileage']),
                    int(row['Gears']),
                    row['Power_steering'],
                    str(row['Keyless_entry']).lower() in ['true', '1', 'yes'],
                    float(row['power_hp']),
                    float(row['torque_nm']),
                    int(row['Seating_capacity']),
                    row['Transmission_type'],
                    int(row['warranty']),
                    str(row['Sun_visor']).lower() in ['true', '1', 'yes'],
                    str(row['Ventilation_system']).lower() in ['true', '1', 'yes'],
                    str(row['ABS']).lower() in ['true', '1', 'yes'],
                    str(row['Airbags']).lower() in ['true', '1', 'yes'],
                    row['color']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### make the sales table

In [47]:
# Drop the sales table if it exists
cursor.execute("DROP TABLE IF EXISTS sales")

# Create the sales table with the Date column and Car_ID as a foreign key to cars
cursor.execute("""
    CREATE TABLE sales (
        dealer_username VARCHAR(100),
        customer_username VARCHAR(100),
        brand_id VARCHAR(255),
        car_id VARCHAR(255),
        sales_id VARCHAR(20) PRIMARY KEY,
        date DATETIME,
        FOREIGN KEY (brand_id) REFERENCES car_brands(brand_id),
        FOREIGN KEY (dealer_username) REFERENCES dealers(username),
        FOREIGN KEY (customer_username) REFERENCES customers(username),
        FOREIGN KEY (car_id) REFERENCES cars(car_id)
    )
""")

In [48]:
try:
    df_sales = pd.read_excel('.\\data\\sales.xlsx')
    for _, row in df_sales.iterrows():
        # Check if brand_id exists in car_brands
        cursor.execute("SELECT 1 FROM car_brands WHERE brand_id = %s", (row['brandId'],))
        if cursor.fetchone() is None:
            print(f"Warning: brand_id '{row['brandId']}' does not exist in car_brands. Skipping row: {row.to_dict()}")
            continue
        # Check if car_id exists in cars
        cursor.execute("SELECT 1 FROM cars WHERE car_id = %s", (row['CarId'],))
        if cursor.fetchone() is None:
            print(f"Warning: car_id '{row['CarId']}' does not exist in cars. Skipping row: {row.to_dict()}")
            continue
        # Check if dealer_username exists in dealers
        cursor.execute("SELECT 1 FROM dealers WHERE username = %s", (row['DealerUsername'],))
        if cursor.fetchone() is None:
            print(f"Warning: dealer_username '{row['DealerUsername']}' does not exist in dealers. Skipping row: {row.to_dict()}")
            continue
        # Check if customer_username exists in customers
        cursor.execute("SELECT 1 FROM customers WHERE username = %s", (row['customerUsername'],))
        if cursor.fetchone() is None:
            print(f"Warning: customer_username '{row['customerUsername']}' does not exist in customers. Skipping row: {row.to_dict()}")
            continue
        try:
            cursor.execute(
                """
                INSERT INTO sales (
                    dealer_username, customer_username, brand_id, car_id, sales_id, date
                ) VALUES (
                    %s, %s, %s, %s, %s, %s
                )
                """,
                (
                    row['DealerUsername'],
                    row['customerUsername'],
                    row['brandId'],
                    row['CarId'],
                    row['sales_ID'],
                    row['sales_date']
                )
            )
        except Exception as e:
            print(f"Error inserting row {row.to_dict()}: {e}")
    conn.commit()
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")


### add status column in the cars table

In [49]:
# 1. Add the status column to the cars table if it doesn't exist
try:
    cursor.execute("ALTER TABLE cars ADD COLUMN status VARCHAR(20) DEFAULT 'AVAILABLE' NOT NULL")
except Exception as e:
    # Ignore error if column already exists
    if "Duplicate column name" not in str(e):
        print(f"Error adding status column: {e}")

# 2. Set all cars to 'Available' by default
cursor.execute("UPDATE cars SET status = 'AVAILABLE'")

# 3. Update status to 'Sold' for cars present in the sales table
cursor.execute("UPDATE cars SET status = 'SOLD' WHERE car_id IN (SELECT car_id FROM sales)")

# 4. Commit the changes
conn.commit()
print("Status column updated in cars table.")

Status column updated in cars table.


### Some updates as per aditya said 

In [50]:
# Update all color values in the cars table to upper case
cursor.execute("UPDATE cars SET color = UPPER(color)")
conn.commit()
print("All colors in the cars table have been converted to upper case.")

All colors in the cars table have been converted to upper case.


In [51]:
cursor.execute("alter table cars add column car_image varchar(255) default 'https://formbuilder.ccavenue.com/live/uploads/company_image/488/17316704336156_Event-Image-Not-Found.jpg' not null")
conn.commit()
print("Image URL column added to cars table with default value.")

Image URL column added to cars table with default value.


In [52]:
cursor.execute("alter table dealers add column profile_image varchar(255) default 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS59s6qBOFlkS5LN4Z0U3G71nCWWg3SuHGVMw&s' not null")
conn.commit()
print("Profile image column added to dealers table with default value.")

Profile image column added to dealers table with default value.
